In [1]:
import credentials  # Import api/access_token keys from credentials.py
import settings  # Import related setting constants from settings.py
from nltk.stem import WordNetLemmatizer
import re
from textblob import TextBlob
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
import tweepy
import pickle
import mysql.connector
import joblib

In [7]:
#!pip install tweepy
#!pip install textblob
#!pip install keras
#!pip install pickle
!pip install mysql-connector-python
#!pip install joblib
#!pip install tensorflow

     |████████████████████████████████| 15.8 MB 2.8 MB/s eta 0:00:01


In [2]:
model = load_model('model.h5')
tokenizer = joblib.load('tokenizer.sav')

In [3]:
class MyStreamListener(tweepy.StreamListener):

    def on_status(self, status):
        '''
        Extract info from tweets
        '''
        try:
            if status.retweeted:
                # Avoid retweeted info, and only original tweets will be received
                return True
            # Extract attributes from each tweet
            id_str = status.id_str
            created_at = status.created_at
            text = self.lemitizor(status.text)  # Pre-processing the text
            sentiment = predict(text)
            user_created_at = status.user.created_at
            user_location = self.deEmojify(status.user.location)
            user_description = self.deEmojify(status.user.description)
            user_followers_count = status.user.followers_count
            longitude = None
            latitude = None
            if status.coordinates:
                longitude = status.coordinates['coordinates'][0]
                latitude = status.coordinates['coordinates'][1]

            retweet_count = status.retweet_count
            favorite_count = status.favorite_count

            # Store all data in MySQL
            if mydb.is_connected():
                mycursor = mydb.cursor()
                sql = "INSERT INTO {} (id_str, created_at, text, sentiment, user_created_at, user_location, user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(
                    settings.TABLE_NAME)
                val = (id_str, created_at, text, sentiment, user_created_at, user_location,user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count)
                mycursor.execute(sql, val)
                mydb.commit()
                mycursor.close()
                print(text)
                print("Long: {}, Lati: {}".format(longitude, latitude))

        except:
            pass

    def on_error(self, status_code):
        '''
        Since Twitter API has rate limits, stop srcraping data as it exceed to the thresold.
        '''
        if status_code == 420:
            # return False to disconnect the stream
            return False

    def clean_tweet(self, tweet):
        '''
        Use sumple regex statemnents to clean tweet text by removing links and special characters
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

    def deEmojify(self,text):
        '''
        Strip all non-ASCII characters to remove emoji characters
        '''
        if text:
            return text.encode('ascii', 'ignore').decode('ascii')
        else:
            return None

    def lemitizor(self, text):
        try:
            lemet = WordNetLemmatizer()
            if text:
                text = self.clean_tweet(text)
                text = text.replace('RT','')
                text = self.deEmojify(text)
                tokenized_tweet = text.split()
                lemetized_tweet = []
                for i in tokenized_tweet:
                    lemetized_tweet.append(lemet.lemmatize(i)) # lemmitizing
                return ' '.join(lemetized_tweet)
            else:
                return None
        except:
            return None

In [4]:
def decode_sentiment(score, include_neutral=True):
        SENTIMENT_THRESHOLDS = (0.4, 0.7)
        if include_neutral:
            label = 0
            if score <= SENTIMENT_THRESHOLDS[0]:
                label = -1
            elif score >= SENTIMENT_THRESHOLDS[1]:
                label = 1

            return label
        else:
            return -1 if score < 0.5 else 1

def predict(text, include_neutral=True):
        SEQUENCE_LENGTH = 300
        x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=SEQUENCE_LENGTH)
        # Predict
        score = model.predict([x_test])[0]
        # Decode sentiment
        label = decode_sentiment(score, include_neutral=include_neutral)
        return label

In [ ]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    database="twitterdb",
    charset = 'utf8'
)
if mydb.is_connected():
    '''
    Check if this table exits. If not, then create a new one.
    '''
    mycursor = mydb.cursor()
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(settings.TABLE_NAME))
    if mycursor.fetchone()[0] != 1:
        mycursor.execute("CREATE TABLE {} ({})".format(settings.TABLE_NAME, settings.TABLE_ATTRIBUTES))
        mydb.commit()
    mycursor.close()

auth = tweepy.OAuthHandler(credentials.API_KEY, credentials.API_SECRET_KEY)
auth.set_access_token(credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener = myStreamListener)
myStream.filter(languages=["en"], track = settings.TRACK_WORDS,locations=[68.1766451354, 7.96553477623, 97.4025614766, 35.4940095078])
mydb.close()